In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 📚 에이전트 메모리 관리하기

결국 에이전트는 간단한 구성 요소로 정의할 수 있습니다. 도구와 프롬프트가 있습니다.
그리고 가장 중요한 것은 계획, 실행 및 오류의 이력을 그리는 과거 단계의 메모리가 있다는 것입니다.

### 에이전트 메모리 재생하기

과거 에이전트 실행을 검사하기 위한 몇 가지 기능을 제안합니다.

[계측 가이드](https://huggingface.co/docs/smolagents/main/ko/tutorials/./inspect_runs)에서 강조 표시된 것처럼 특정 단계를 확대/축소할 수 있는 훌륭한 UI에 에이전트 실행을 표시하도록 계측할 수 있습니다.

다음과 같이 `agent.replay()`를 사용할 수도 있습니다.

에이전트 실행 후:

In [ ]:
from smolagents import InferenceClientModel, CodeAgent

agent = CodeAgent(tools=[], model=InferenceClientModel(), verbosity_level=0)

result = agent.run("20번째 피보나치 수는 무엇인가요?")

이 마지막 실행을 재생하려면 다음을 사용하십시오.

In [ ]:
agent.replay()

### 에이전트 메모리 동적으로 변경하기

많은 고급 사용 사례에는 에이전트 메모리의 동적 수정이 필요합니다.

다음을 사용하여 에이전트 메모리에 액세스할 수 있습니다.

In [ ]:
from smolagents import ActionStep

system_prompt_step = agent.memory.system_prompt
print("에이전트에 제공된 시스템 프롬프트는 다음과 같습니다.")
print(system_prompt_step.system_prompt)

task_step = agent.memory.steps[0]
print("\n\n첫 번째 작업 단계는 다음과 같습니다.")
print(task_step.task)

for step in agent.memory.steps:
    if isinstance(step, ActionStep):
        if step.error is not None:
            print(f"\n{step.step_number}단계에서 다음 오류가 발생했습니다.\n{step.error}\n")
        else:
            print(f"\n{step.step_number}단계에서 다음 관찰 결과를 얻었습니다.\n{step.observations}\n")

전체 단계를 사전으로 가져오려면 `agent.memory.get_full_steps()`를 사용하십시오.

단계 콜백을 사용하여 에이전트 메모리를 동적으로 변경할 수도 있습니다.

단계 콜백은 인수에서 `agent` 자체에 액세스할 수 있으므로 위에서 강조 표시된 것처럼 모든 메모리 단계에 액세스하고 필요한 경우 변경할 수 있습니다. 예를 들어 웹 브라우저 에이전트가 수행한 각 단계의 스크린샷을 관찰하고 있다고 가정해 보겠습니다. 최신 스크린샷을 기록하고 토큰 비용을 절약하기 위해 이전 단계의 이미지를 제거하려고 합니다.

다음과 유사한 코드를 실행할 수 있습니다.
_참고: 이 코드는 불완전하며 간결성을 위해 일부 가져오기 및 개체 정의가 제거되었습니다. 전체 작동 코드를 얻으려면 [원본 스크립트](https://github.com/huggingface/smolagents/blob/main/src/smolagents/vision_web_browser.py)를 방문하십시오._

In [ ]:
import helium
from PIL import Image
from io import BytesIO
from time import sleep

def update_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # 스크린샷을 찍기 전에 JavaScript 애니메이션이 발생하도록 합니다.
    driver = helium.get_driver()
    latest_step = memory_step.step_number
    for previous_memory_step in agent.memory.steps:  # 간결한 처리를 위해 로그에서 이전 스크린샷 제거
        if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= latest_step - 2:
            previous_memory_step.observations_images = None
    png_bytes = driver.get_screenshot_as_png()
    image = Image.open(BytesIO(png_bytes))
    memory_step.observations_images = [image.copy()]

그런 다음 에이전트 초기화 시 `step_callbacks` 인수에 이 함수를 전달해야 합니다.

In [ ]:
CodeAgent(
    tools=[WebSearchTool(), go_back, close_popups, search_item_ctrl_f],
    model=model,
    additional_authorized_imports=["helium"],
    step_callbacks=[update_screenshot],
    max_steps=20,
    verbosity_level=2,
)

전체 작동 예제를 보려면 [비전 웹 브라우저 코드](https://github.com/huggingface/smolagents/blob/main/src/smolagents/vision_web_browser.py)로 이동하십시오.

### 한 번에 한 단계씩 에이전트 실행하기

며칠이 걸리는 도구 호출이 있는 경우 유용할 수 있습니다. 에이전트를 단계별로 실행하기만 하면 됩니다.
이렇게 하면 각 단계에서 메모리를 업데이트할 수도 있습니다.

In [ ]:
from smolagents import InferenceClientModel, CodeAgent, ActionStep, TaskStep

agent = CodeAgent(tools=[], model=InferenceClientModel(), verbosity_level=1)
agent.python_executor.send_tools({**agent.tools})
print(agent.memory.system_prompt)

task = "20번째 피보나치 수는 무엇인가요?"

# 다른 에이전트의 메모리를 입력하여 여기서 필요에 따라 메모리를 수정할 수 있습니다.
# agent.memory.steps = previous_agent.memory.steps

# 새 작업을 시작합시다!
agent.memory.steps.append(TaskStep(task=task, task_images=[]))

final_answer = None
step_number = 1
while final_answer is None and step_number <= 10:
    memory_step = ActionStep(
        step_number=step_number,
        observations_images=[],
    )
    # 한 단계 실행합니다.
    final_answer = agent.step(memory_step)
    agent.memory.steps.append(memory_step)
    step_number += 1

    # 원하는 대로 메모리를 변경하십시오!
    # 예를 들어 최신 단계를 업데이트하려면:
    # agent.memory.steps[-1] = ...

print("최종 답변은 다음과 같습니다.", final_answer)